# 로컬런타임 연결

In [ ]:
"""로컬 런타임 연결(터미널)
activate tensorflow

jupyter notebook \
    --NotebookApp.allow_origin='https://colab.research.google.com' \
    --port=8888 \
    --NotebookApp.port_retries=0
"""

"로컬 런타임 연결(터미널)\nactivate tensorflow\n\njupyter notebook     --NotebookApp.allow_origin='https://colab.research.google.com'     --port=8888     --NotebookApp.port_retries=0\n"

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv
import matplotlib.pyplot as plt
import torch

print(torch.__version__) #토치 버전 '2.2.2+cu118'
print(torch.version.cuda) #CUDA 버전
print(torch.cuda.get_device_name())
print(torch.cuda.device_count())
print(torch.cuda.is_available())

2.3.0+cu118
11.8
NVIDIA GeForce MX450
1
True


In [2]:
import tensorflow as tf

# Print the installed version of TensorFlow
print("TensorFlow version:", tf.__version__)

# List all available devices detected by TensorFlow
print("Available devices:")
devices = tf.config.list_physical_devices()
for device in devices:
    print(device)

# Check if TensorFlow is currently using a GPU
gpu_available = tf.config.list_physical_devices('GPU')
if gpu_available:
    print("GPU is available for TensorFlow.")
else:
    print("GPU is not available, TensorFlow is using CPU.")

TensorFlow version: 2.10.0
Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU is available for TensorFlow.


# 31 classes 분류

데이터셋 정규화 및 증강

In [37]:
import os
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, ConcatDataset
import torch
from tqdm import tqdm  # tqdm 불러오기

# Train 데이터셋 경로 (processed_images와 sorted_training_images)
train_dir_1 = 'C:/Users/SW/Desktop/데크캠/2024 데이터 크리에이터 캠프 대학부/dataset/processed_images'
train_dir_2 = 'C:/Users/SW/Desktop/데크캠/2024 데이터 크리에이터 캠프 대학부/dataset/sorted_training_images'

# Val 데이터 경로 설정
val_dir = 'C:/Users/SW/Desktop/데크캠/2024 데이터 크리에이터 캠프 대학부/dataset/sorted_validation_images'

# 이미지 로드 및 기본 전처리 (이미지 크기만 일단 조정)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# train 원본 데이터만 확인
train_dataset_2 = datasets.ImageFolder(root=train_dir_2, transform=transform)

# DataLoader 설정
batch_size = 32
origin_train_loader = DataLoader(train_dataset_2, batch_size=batch_size, shuffle=True, num_workers=4)

# 평균과 표준편차 계산 함수에 tqdm 추가
def calculate_mean_std(loader):
    mean = torch.zeros(3)
    std = torch.zeros(3)
    total_images_count = 0

    # tqdm을 사용하여 진행 상황을 보여줌
    for images, _ in tqdm(loader, desc="Calculating mean and std"):
        batch_samples = images.size(0)  # 배치 내 이미지 개수
        images = images.view(batch_samples, images.size(1), -1)  # [batch_size, channels, height*width]
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count

    return mean, std

# 평균과 표준편차 계산
print("Calculating for train dataset...")
mean, std = calculate_mean_std(origin_train_loader)
print(f"Train Mean: {mean}")
print(f"Train Standard Deviation: {std}")

Calculating for train dataset...


Calculating mean and std: 100%|██████████████████████████████████████████████████████| 128/128 [15:43<00:00,  7.37s/it]

Train Mean: tensor([0.5498, 0.5226, 0.5052])
Train Standard Deviation: tensor([0.2600, 0.2582, 0.2620])


각 클래스별 데이터 수에 대한 가중치 계산

In [38]:
from collections import Counter
import torch

# 클래스별 샘플 수 계산
def get_class_weights(origin_train_loader):
    targets = []
    for _, labels in origin_train_loader:
        targets.extend(labels.numpy())  # 모든 레이블 수집

    class_counts = Counter(targets)  # 각 클래스별 샘플 수
    total_samples = sum(class_counts.values())  # 전체 샘플 수

    # 샘플 수에 반비례하는 가중치 계산
    class_weights = {cls: total_samples / count for cls, count in class_counts.items()}

    # 가중치 배열을 torch.Tensor로 변환
    weights = torch.tensor([class_weights[i] for i in range(len(class_counts))], dtype=torch.float)

    return weights

# 클래스 가중치 계산
class_weights = get_class_weights(origin_train_loader)

데이터셋 전처리 및 로드

In [39]:
import os
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, ConcatDataset

# 이미지 로드 및 기본 전처리 + 정규화
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5498, 0.5226, 0.5052], std=[0.2600, 0.2582, 0.2620])  # 계산한 평균, 표준편차를 활용한 정규화
])

# 증강을 추가한 transform
augment_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=1),  # 좌우 반전
    transforms.ColorJitter(saturation=0.3, brightness=0.3),  # 채도와 밝기 변환
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5498, 0.5226, 0.5052], std=[0.2600, 0.2582, 0.2620])
])

# 각 경로의 데이터셋 불러오기
train_dataset_1 = datasets.ImageFolder(root=train_dir_1, transform=transform)
train_dataset_2 = datasets.ImageFolder(root=train_dir_2, transform=transform)
# 증강된 이미지 데이터셋
train_dataset_2_augmented = datasets.ImageFolder(root=train_dir_2, transform=augment_transform)

# 세 개의 데이터셋을 합침
train_dataset = ConcatDataset([train_dataset_1, train_dataset_2, train_dataset_2_augmented])

# validation 데이터셋
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

# DataLoader 설정
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


print(f"Train size: {len(train_loader.dataset)}, Val size: {len(val_loader.dataset)}")

Train size: 12210, Val size: 951


모델 하이퍼파라미터

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Hyperparameters
num_epochs = 100
learning_rate = 0.01

# Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ResNet-18 모델 정의
model = models.resnet18(weights=None)  # pretrained=False로 설정
num_features = model.fc.in_features  # 마지막 FC 레이어의 입력 크기
model.fc = nn.Linear(num_features, 31)  # 출력 클래스 수에 맞게 수정
model = model.to(device)

# Loss 및 Optimizer 설정
criterion = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))  # 가중치를 이용한 손실 함수 정의
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)  # 가중치 크기 제한

# ReduceLROnPlateau 스케줄러 초기화
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, min_lr=1e-6)

Using device: cuda


진행상황 저장

In [29]:
# 체크포인트 저장 함수
def save_checkpoint(state, filename="C:/Users/SW/Desktop/데크캠/mission1-2/checkpoint_3rd.pth", weights_only=False):
    torch.save(state, filename)


# 체크포인트 로드 함수
def load_checkpoint(filename="C:/Users/SW/Desktop/데크캠/mission1-2/checkpoint_3rd.pth", weights_only=False):
    checkpoint = torch.load(filename)
    epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    best_loss = checkpoint['best_loss']
    return epoch, best_loss

resnet-18 학습모델

In [42]:
# 학습 및 검증 함수
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, start_epoch=0, best_loss=float('inf')):
    history = {'train_loss': [], 'train_accuracy': [], 'val_loss': [], 'val_accuracy': []}

    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Accuracy 계산
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        avg_loss = running_loss / len(train_loader)
        accuracy = correct_predictions / total_samples
        history['train_loss'].append(avg_loss)
        history['train_accuracy'].append(accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.4f}")

        # Validation phase
        model.eval()
        val_running_loss = 0.0
        val_correct_predictions = 0
        val_total_samples = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                val_total_samples += labels.size(0)
                val_correct_predictions += (predicted == labels).sum().item()

        avg_val_loss = val_running_loss / len(val_loader)
        val_accuracy = val_correct_predictions / val_total_samples
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(val_accuracy)

        print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        # ReduceLROnPlateau 스케줄러를 사용하여 학습률 조정
        scheduler.step(avg_val_loss)
        current_lr = scheduler.get_last_lr()[0]  # 현재 학습률 확인
        print(f"Current Learning Rate: {current_lr}")

        # 가장 낮은 loss를 가진 모델 저장
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')

            # 체크포인트 저장
            save_checkpoint({
                'epoch': epoch + 1,  # 현재 에폭 저장
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_loss': best_loss
            })

    return history


In [45]:
# 만약 기존 체크포인트가 있다면 로드
try:
    start_epoch, best_loss = load_checkpoint()
    print(f"Checkpoint loaded. Resuming from epoch {start_epoch} with lowest loss {best_loss:.4f}.")
except FileNotFoundError:
    print("No checkpoint found, starting from scratch.")
    start_epoch, best_loss = 0, 10.0

# 학습 재개
history = train_and_validate(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, start_epoch, best_loss)

No checkpoint found, starting from scratch.


Training Epoch 1/100: 100%|██████████████████████████████████████████████████████████| 382/382 [21:34<00:00,  3.39s/it]


Epoch [1/100], Train Loss: 3.5487, Train Accuracy: 0.0369
Validation Loss: 3.4628, Validation Accuracy: 0.0515
Current Learning Rate: 0.01


Training Epoch 2/100: 100%|██████████████████████████████████████████████████████████| 382/382 [21:04<00:00,  3.31s/it]


Epoch [2/100], Train Loss: 3.4360, Train Accuracy: 0.0387
Validation Loss: 3.4321, Validation Accuracy: 0.0389
Current Learning Rate: 0.01


...

Epoch [50/100], Train Loss: 0.0033, Train Accuracy: 0.9993
Validation Loss: 2.9289, Validation Accuracy: 0.6141
Current Learning Rate: 0.0003125


Training Epoch 51/100: 100%|█████████████████████████████████████████████████████████| 382/382 [21:44<00:00,  3.42s/it]


Epoch [51/100], Train Loss: 0.0031, Train Accuracy: 0.9993
Validation Loss: 2.9588, Validation Accuracy: 0.6141
Current Learning Rate: 0.00015625


Training Epoch 52/100:  16%|█████████                                                 | 60/382 [04:24<23:37,  4.40s/it]


KeyboardInterrupt: 